<pre>
Copyright 2022-2025 Boris Shminke

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
</pre>

In [1]:
# since both Jupyter and `isabelle-client` use `asyncio` we need to
# enable nested event loops. We don't need that when using
# `isabelle-client` from Python scripts outside Jupyter
import nest_asyncio

nest_asyncio.apply()

In [2]:
# first, we start Isabelle server
from isabelle_client import start_isabelle_server

server_info, isabelle_process = start_isabelle_server()

In [3]:
# server info is the same as printed by `isabelle server` command
print(server_info)

server "isabelle" = 127.0.0.1:44197 (password "6c1073ce-9519-49d9-a8c6-84ca39d44026")



In [4]:
# a process object can be used for additional control
type(isabelle_process)

asyncio.subprocess.Process

In [5]:
isabelle_process

<Process 1191>

In [6]:
isabelle_process.pid

1191

In [7]:
# now we can create a client instance
from isabelle_client import get_isabelle_client

isabelle = get_isabelle_client(server_info)
isabelle

In [8]:
# the server's commands are implemented as client object methods
responses = isabelle.help()
responses

[HelpResult(response_type=<IsabelleResponseType.OK: 'OK'>, response_body=['cancel', 'echo', 'help', 'purge_theories', 'session_build', 'session_start', 'session_stop', 'shutdown', 'use_theories'], response_length=118)]

In [9]:
# usually, the last server's response contains something interesting
responses[-1]

HelpResult(response_type=<IsabelleResponseType.OK: 'OK'>, response_body=['cancel', 'echo', 'help', 'purge_theories', 'session_build', 'session_start', 'session_stop', 'shutdown', 'use_theories'], response_length=118)

In [10]:
responses[-1].response_body

['cancel',
 'echo',
 'help',
 'purge_theories',
 'session_build',
 'session_start',
 'session_stop',
 'shutdown',
 'use_theories']

In [11]:
%%time
# let's do something more meaningful

session_start_result = isabelle.session_start()
session_id = session_start_result[-1].response_body.session_id

CPU times: user 1 ms, sys: 3.05 ms, total: 4.05 ms
Wall time: 4.15 s


In [12]:
# let's generate a dummy problem file in Isabelle format
from pathlib import Path

problem_text = r"""
theory Problem imports Main
begin
lemma "\<forall> x. \<exists> y. x = y"
sledgehammer
oops
end
"""
Path("Problem.thy").write_text(problem_text)

97

In [13]:
%%time
# now let's ask the server to solve the problem

responses = isabelle.use_theories(
    theories=["Problem"], master_dir=".", session_id=session_id
)
responses

CPU times: user 2.7 ms, sys: 235 μs, total: 2.94 ms
Wall time: 2.32 s


[TaskOK(response_type=<IsabelleResponseType.OK: 'OK'>, response_body=Task(task='d649dde6-103f-48d3-9b40-1e3e079fa523'), response_length=None),
 TaskOK(response_type=<IsabelleResponseType.NOTE: 'NOTE'>, response_body=Task(task='d649dde6-103f-48d3-9b40-1e3e079fa523'), response_length=161),
 TaskOK(response_type=<IsabelleResponseType.NOTE: 'NOTE'>, response_body=Task(task='d649dde6-103f-48d3-9b40-1e3e079fa523'), response_length=161),
 TaskOK(response_type=<IsabelleResponseType.NOTE: 'NOTE'>, response_body=Task(task='d649dde6-103f-48d3-9b40-1e3e079fa523'), response_length=163),
 UseTheoriesResponse(response_type=<IsabelleResponseType.FINISHED: 'FINISHED'>, response_body=UseTheoriesResults(ok=True, errors=[], nodes=[NodeResult(node_name='Problem.thy', theory_name='Draft.Problem', status=NodeStatus(ok=True, total=11, unprocessed=0, running=0, warned=0, failed=0, finished=11, canceled=False, consolidated=True, percentage=100), messages=[Message(kind='writeln', message='Sledgehammering...', po

In [14]:
# notice that important messages live inside "nodes" object
responses[-1].response_body

UseTheoriesResults(ok=True, errors=[], nodes=[NodeResult(node_name='Problem.thy', theory_name='Draft.Problem', status=NodeStatus(ok=True, total=11, unprocessed=0, running=0, warned=0, failed=0, finished=11, canceled=False, consolidated=True, percentage=100), messages=[Message(kind='writeln', message='Sledgehammering...', pos=Position(line=5, offset=60, end_offset=72, file='Problem.thy', id=None)), Message(kind='writeln', message='verit found a proof...', pos=Position(line=5, offset=60, end_offset=72, file='Problem.thy', id=None)), Message(kind='writeln', message='zipperposition found a proof...', pos=Position(line=5, offset=60, end_offset=72, file='Problem.thy', id=None)), Message(kind='writeln', message='cvc5 found a proof...', pos=Position(line=5, offset=60, end_offset=72, file='Problem.thy', id=None)), Message(kind='writeln', message='zipperposition found a proof...', pos=Position(line=5, offset=60, end_offset=72, file='Problem.thy', id=None)), Message(kind='writeln', message='vampi

In [15]:
# the messages from `sledgehammer` have an easily parsable format
messages = [
    message.message
    for message in responses[-1].response_body.nodes[0].messages
]
messages

['Sledgehammering...',
 'verit found a proof...',
 'zipperposition found a proof...',
 'cvc5 found a proof...',
 'zipperposition found a proof...',
 'vampire found a proof...',
 'verit: Try this: by simp (1 ms)',
 'zipperposition: Duplicate proof',
 'cvc5: Try this: by blast (0.4 ms)',
 'zipperposition: Duplicate proof',
 'vampire: Duplicate proof',
 'spass found a proof...',
 'spass: Duplicate proof',
 'Done']

In [16]:
# we can parse `sledgehammer` suggestions using regular expressions
import re

proposed_solutions = [
    re.findall(r".*: Try this: (.*) \(\d+\.?\d* m?s\)", message)
    for message in messages
]
proposed_solutions

[[], [], [], [], [], [], ['by simp'], [], ['by blast'], [], [], [], [], []]

In [17]:
# in principle, we can value some solutions more
import random

solution = random.choice([
    proposed_solution[0]
    for proposed_solution in proposed_solutions
    if proposed_solution
])
solution

'by simp'

In [18]:
# now let's write down our solution to a file
solution_text = problem_text.replace(
    "sledgehammer\noops\n", f"{solution}\n"
).replace("theory Problem ", "theory Solution ")
print(solution_text)
Path("Solution.thy").write_text(solution_text)


theory Solution imports Main
begin
lemma "\<forall> x. \<exists> y. x = y"
by simp
end



88

In [19]:
%%time
# and finally let's ask the server to check the solution

responses = isabelle.use_theories(
    theories=["Solution"], master_dir=".", session_id=session_id
)
responses

CPU times: user 2.4 ms, sys: 200 μs, total: 2.6 ms
Wall time: 2.01 s


[TaskOK(response_type=<IsabelleResponseType.OK: 'OK'>, response_body=Task(task='8d55b15b-29e2-48ad-980f-4569d52e4004'), response_length=None),
 TaskOK(response_type=<IsabelleResponseType.NOTE: 'NOTE'>, response_body=Task(task='8d55b15b-29e2-48ad-980f-4569d52e4004'), response_length=163),
 TaskOK(response_type=<IsabelleResponseType.NOTE: 'NOTE'>, response_body=Task(task='8d55b15b-29e2-48ad-980f-4569d52e4004'), response_length=163),
 UseTheoriesResponse(response_type=<IsabelleResponseType.FINISHED: 'FINISHED'>, response_body=UseTheoriesResults(ok=True, errors=[], nodes=[NodeResult(node_name='Solution.thy', theory_name='Draft.Solution', status=NodeStatus(ok=True, total=9, unprocessed=0, running=0, warned=0, failed=0, finished=9, canceled=False, consolidated=True, percentage=100), messages=[Message(kind='writeln', message='theorem \\<forall>x. \\<exists>y. x = y', pos=Position(line=5, offset=61, end_offset=63, file='Solution.thy', id=None))], exports=[])]), response_length=461)]

In [20]:
# the response body doesn't contain any errors
responses[-1].response_body.errors

[]

In [21]:
# we can gracefully finish the session
isabelle.session_stop(session_id)

[TaskOK(response_type=<IsabelleResponseType.OK: 'OK'>, response_body=Task(task='3be40c3c-c1fa-45d9-b7e3-be277e949bba'), response_length=None),
 SessionStopRegularResponse(response_type=<IsabelleResponseType.FINISHED: 'FINISHED'>, response_body=SessionStopRegularResult(ok=True, return_code=0, task='3be40c3c-c1fa-45d9-b7e3-be277e949bba', response_type=<IsabelleResponseType.FINISHED: 'FINISHED'>), response_length=None)]

In [22]:
# and shut the server down
isabelle.shutdown()

SimpleIsabelleResponse(response_type=<IsabelleResponseType.OK: 'OK'>)